In [112]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import tfgraphviz as tfg
from tensorboard import notebook

In [113]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [114]:
reset_graph()

n_inputs = 2
n_hidden1 = 2
n_outputs = 2

In [115]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation=="relu":
            return tf.nn.relu(z)
        if activation=="sigmoid":
            return tf.math.sigmoid(z)
        else:
            return z

In [116]:
from sklearn.model_selection import train_test_split

# Number of examples.
N = 1000

x1 = np.random.randint(0,2,N)
x2 = np.random.randint(0,2,N)

y_xor = x1 ^ x2

x1 = x1 + 0.1*np.random.randn(N,)
x2 = x2 + 0.1*np.random.randn(N,)

x_xor = np.zeros((len(x1), 2))
for i in range(len(x1)):
    x_xor[i,0] = x1[i]
    x_xor[i,1] = x2[i]

# Split array into random train and test subsets.
X_train, X_valid, y_train, y_valid = train_test_split(x_xor, y_xor, random_state=23)

In [120]:
#a)
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="sigmoid")
    logits = neuron_layer(hidden1, n_outputs, "outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.1

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [121]:
print('b)')
n_epochs = 40000
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        loss_train = loss.eval(feed_dict={X: X_batch, y: y_batch})
        loss_test = loss.eval(feed_dict={X: X_valid, y: y_valid})
        if epoch % 100 == 0:
            print(epoch, "Train loss:", loss_train, "Test loss:", loss_test)
    save_path = saver.save(sess, "./my_model_finalv1.ckpt")

b)
0 Train loss: 0.75627196 Test loss: 0.780114
100 Train loss: 0.69866675 Test loss: 0.6983298
200 Train loss: 0.68950045 Test loss: 0.6914551
300 Train loss: 0.63579303 Test loss: 0.65702575
400 Train loss: 0.4905203 Test loss: 0.51534486
500 Train loss: 0.15395682 Test loss: 0.16490595
600 Train loss: 0.07326271 Test loss: 0.07252537
700 Train loss: 0.040938858 Test loss: 0.044510074
800 Train loss: 0.03016437 Test loss: 0.031788163
900 Train loss: 0.026076388 Test loss: 0.02465648
1000 Train loss: 0.017030884 Test loss: 0.020129258
1100 Train loss: 0.016878147 Test loss: 0.017012963
1200 Train loss: 0.011982724 Test loss: 0.014739778
1300 Train loss: 0.01001075 Test loss: 0.013012176
1400 Train loss: 0.010740886 Test loss: 0.011653997
1500 Train loss: 0.007985737 Test loss: 0.01055999
1600 Train loss: 0.008321167 Test loss: 0.009658799
1700 Train loss: 0.007176362 Test loss: 0.008904287
1800 Train loss: 0.006182943 Test loss: 0.008262905
1900 Train loss: 0.006172646 Test loss: 0.00

15300 Train loss: 0.00047845612 Test loss: 0.0008739225
15400 Train loss: 0.000494819 Test loss: 0.0008682547
15500 Train loss: 0.00243787 Test loss: 0.0008628015
15600 Train loss: 0.0004597604 Test loss: 0.00085737294
15700 Train loss: 0.0017591239 Test loss: 0.00085207756
15800 Train loss: 0.0003903003 Test loss: 0.0008467627
15900 Train loss: 0.00045938537 Test loss: 0.0008415652
16000 Train loss: 0.00042798772 Test loss: 0.00083625905
16100 Train loss: 0.0014442847 Test loss: 0.0008312368
16200 Train loss: 0.0024367974 Test loss: 0.00082622207
16300 Train loss: 0.00047526913 Test loss: 0.0008212119
16400 Train loss: 0.0016206136 Test loss: 0.00081629393
16500 Train loss: 0.0003248232 Test loss: 0.00081144046
16600 Train loss: 0.0004405087 Test loss: 0.00080662087
16700 Train loss: 0.0024741855 Test loss: 0.00080175424
16800 Train loss: 0.0013906942 Test loss: 0.00079715595
16900 Train loss: 0.00038194365 Test loss: 0.0007925512
17000 Train loss: 0.00041891017 Test loss: 0.000787948

30100 Train loss: 0.0001651643 Test loss: 0.00044625095
30200 Train loss: 0.0025745074 Test loss: 0.00044471788
30300 Train loss: 0.0024785325 Test loss: 0.00044324537
30400 Train loss: 0.000200393 Test loss: 0.00044179373
30500 Train loss: 0.0001745637 Test loss: 0.00044036054
30600 Train loss: 0.00018463875 Test loss: 0.00043889845
30700 Train loss: 0.00029386865 Test loss: 0.00043745583
30800 Train loss: 0.00030791454 Test loss: 0.000436016
30900 Train loss: 0.00017067243 Test loss: 0.00043458518
31000 Train loss: 0.00020650365 Test loss: 0.00043316005
31100 Train loss: 0.0017919163 Test loss: 0.00043178213
31200 Train loss: 0.0001415117 Test loss: 0.0004303797
31300 Train loss: 0.00018432502 Test loss: 0.0004290236
31400 Train loss: 0.00018505716 Test loss: 0.00042763766
31500 Train loss: 0.0016861028 Test loss: 0.0004262726
31600 Train loss: 0.00013564985 Test loss: 0.00042489188
31700 Train loss: 0.0010492189 Test loss: 0.00042353768
31800 Train loss: 0.00018741332 Test loss: 0.0

In [122]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_finalv1.ckpt") # or better, use save_path
    Z = logits.eval(feed_dict={X: X_valid})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_finalv1.ckpt


In [123]:
from sklearn.metrics import accuracy_score
# Calculate and return the accuracy on the test data
accuracy = accuracy_score(y_valid, y_pred)
print('GradientDescentOptimizer accuracy:', accuracy)

GradientDescentOptimizer accuracy: 1.0


In [124]:
reset_graph()

In [126]:
#c)
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="sigmoid")
    logits = neuron_layer(hidden1, n_outputs, "outputs")
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.1

with tf.name_scope("train"):
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [127]:
print('d)')
n_epochs = 40000
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        loss_train = loss.eval(feed_dict={X: X_batch, y: y_batch})
        loss_test = loss.eval(feed_dict={X: X_valid, y: y_valid})
        if epoch % 100 == 0:
            print(epoch, "Train loss:", loss_train, "Test loss:", loss_test)
    save_path = saver.save(sess, "./my_model_finalv2.ckpt")

d)
0 Train loss: 0.6705238 Test loss: 0.67673093
100 Train loss: 0.0024167998 Test loss: 0.0011575341
200 Train loss: 0.00019471091 Test loss: 0.00039884966
300 Train loss: 0.00053543237 Test loss: 0.0001990199
400 Train loss: 0.00032568534 Test loss: 0.00011874849
500 Train loss: 0.00023515559 Test loss: 7.869401e-05
600 Train loss: 1.5319756e-05 Test loss: 5.3570464e-05
700 Train loss: 1.1277878e-05 Test loss: 3.9812243e-05
800 Train loss: 5.028547e-05 Test loss: 3.0242772e-05
900 Train loss: 3.908472e-05 Test loss: 2.3482637e-05
1000 Train loss: 2.9880168e-06 Test loss: 1.8288581e-05
1100 Train loss: 1.7669652e-06 Test loss: 1.4639259e-05
1200 Train loss: 2.1956435e-05 Test loss: 1.1746229e-05
1300 Train loss: 1.07126825e-05 Test loss: 9.463927e-06
1400 Train loss: 6.8897916e-06 Test loss: 7.677755e-06
1500 Train loss: 3.7250086e-06 Test loss: 6.3146417e-06
1600 Train loss: 3.4760072e-07 Test loss: 5.0757008e-06
1700 Train loss: 1.7157218e-07 Test loss: 4.1666904e-06
1800 Train loss

17200 Train loss: 0.0 Test loss: 9.536743e-10
17300 Train loss: 0.0 Test loss: 9.536743e-10
17400 Train loss: 0.0 Test loss: 9.536743e-10
17500 Train loss: 0.0 Test loss: 9.536743e-10
17600 Train loss: 0.0 Test loss: 9.536743e-10
17700 Train loss: 0.0 Test loss: 9.536743e-10
17800 Train loss: 0.0 Test loss: 9.536743e-10
17900 Train loss: 0.0 Test loss: 9.536743e-10
18000 Train loss: 0.0 Test loss: 9.536743e-10
18100 Train loss: 0.0 Test loss: 9.536743e-10
18200 Train loss: 0.0 Test loss: 9.536743e-10
18300 Train loss: 0.0 Test loss: 9.536743e-10
18400 Train loss: 0.0 Test loss: 9.536743e-10
18500 Train loss: 0.0 Test loss: 9.536743e-10
18600 Train loss: 0.0 Test loss: 9.536743e-10
18700 Train loss: 0.0 Test loss: 9.536743e-10
18800 Train loss: 0.0 Test loss: 9.536743e-10
18900 Train loss: 0.0 Test loss: 9.536743e-10
19000 Train loss: 0.0 Test loss: 9.536743e-10
19100 Train loss: 0.0 Test loss: 9.536743e-10
19200 Train loss: 0.0 Test loss: 9.536743e-10
19300 Train loss: 0.0 Test loss: 9

35100 Train loss: 0.0 Test loss: 4.768371e-10
35200 Train loss: 0.0 Test loss: 4.768371e-10
35300 Train loss: 0.0 Test loss: 4.768371e-10
35400 Train loss: 0.0 Test loss: 4.768371e-10
35500 Train loss: 0.0 Test loss: 4.768371e-10
35600 Train loss: 0.0 Test loss: 4.768371e-10
35700 Train loss: 0.0 Test loss: 4.768371e-10
35800 Train loss: 0.0 Test loss: 4.768371e-10
35900 Train loss: 0.0 Test loss: 4.768371e-10
36000 Train loss: 0.0 Test loss: 4.768371e-10
36100 Train loss: 0.0 Test loss: 4.768371e-10
36200 Train loss: 0.0 Test loss: 4.768371e-10
36300 Train loss: 0.0 Test loss: 4.768371e-10
36400 Train loss: 0.0 Test loss: 4.768371e-10
36500 Train loss: 0.0 Test loss: 4.768371e-10
36600 Train loss: 0.0 Test loss: 4.768371e-10
36700 Train loss: 0.0 Test loss: 4.768371e-10
36800 Train loss: 0.0 Test loss: 4.768371e-10
36900 Train loss: 0.0 Test loss: 4.768371e-10
37000 Train loss: 0.0 Test loss: 4.768371e-10
37100 Train loss: 0.0 Test loss: 4.768371e-10
37200 Train loss: 0.0 Test loss: 4

In [128]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_finalv2.ckpt") # or better, use save_path
    Z = logits.eval(feed_dict={X: X_valid})
    y_pred = np.argmax(Z, axis=1)

INFO:tensorflow:Restoring parameters from ./my_model_finalv2.ckpt


In [129]:
from sklearn.metrics import accuracy_score
# Calculate and return the accuracy on the test data
accuracy = accuracy_score(y_valid, y_pred)
print('AdamOptimizer accuracy:', accuracy)

AdamOptimizer accuracy: 1.0


## Ambos os otimizadores tem a maior precisão após 40000 epochs, mas o Adam atinge um menor error em menos epochs que o Gradient Descent portanto o otimizador Adam tem melhor performance